## Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from collections.abc import Iterable

## Custom Plots

In [3]:
def custom_subplots(*args, facecolor='#FFFFFFFF', size=None, **kwargs):
    """
    Create a custom subplot with specified facecolor and size.

    Parameters:
    - *args: Positional arguments for plt.subplots.
    - facecolor: Background color of the subplot.
    - size: Tuple specifying the size of the figure (width, height).
    - **kwargs: Additional keyword arguments for plt.subplots.

    Returns:
    - fig: The created figure.
    - axes: The created axes.
    """

    # Set full-width default if size is not given
    if size is None:
        size = (16, 4)

    fig, axes = plt.subplots(*args, **kwargs)
    fig.set_size_inches(size)

    if not (isinstance(axes, Iterable) and not isinstance(axes, np.ndarray) or (isinstance(axes, np.ndarray) and axes.ndim > 0)):
        axes = [axes]
      
    for ax in np.ravel(axes):
        ax.set_xticks([])
        ax.set_yticks([])

        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1)

    plt.subplots_adjust(wspace=0.05)
    fig.patch.set_facecolor(facecolor)

    if len(np.ravel(axes)) == 1:
        axes = np.ravel(axes)[0]
    else:
        axes = np.ravel(axes)

    return fig, axes

## Rendering hyperspectral images as RGB

In [4]:
# Clip the top and bottom 2% of the data to not normalise to outliers.
def clip_to_quantiles(arr: np.ndarray, q_min: float = 0.02, q_max: float = 0.98) -> np.ndarray:
  """
  Clip the array to the specified quantiles.

  Args:
    arr (np.ndarray): The input array to clip.
    q_min (float): The lower quantile to clip to.
    q_max (float): The upper quantile to clip to.

  Returns:
    np.ndarray: The clipped array.
  """
  return np.clip(
    arr,
    np.nanquantile(arr, q_min),
    np.nanquantile(arr, q_max)
  )

def render_s2_as_rgb(arr: np.ndarray) -> np.ndarray:
  """
  Render Sentinel-2 data as RGB by clipping and normalising the bands.

  Args:
    arr (np.ndarray): The input Sentinel-2 data array with shape (height, width, channels).
  
  Returns:
    np.ndarray: The 8-bit RGB rendered array with shape (height, width, 3).
  """
  # If there are no data values, cast them to zero
  if np.ma.isMaskedArray(arr):
    arr = np.ma.getdata(arr.filled(0))
  
  # Select only blue, green and red bands
  rgb_slice = arr[:, :, 0:3]

  # Clip the data to the quantiles, so the RGB render is not stretched to outliers, which produces dark images
  for c in range(3):
    rgb_slice[:, :, c] = clip_to_quantiles(rgb_slice[:, :, c])
  
  # The current slice is uint16, but we want an uint8 RGB render
  # We normalise the layer by dividing with the maximum value in the image
  # Then we multiply it by 255 (the max of uint8) to get the normal RGB range
  for c in range(3):
    rgb_slice[:, :, c] = (rgb_slice[:, :, c] / np.max(rgb_slice[:, :, c])) * 255.0
  
  # We then round to the nearest integer and cast it to uint8
  rgb_slice = np.rint(rgb_slice).astype(np.uint8)

  return rgb_slice